1. **Configuring Access to Polytope**

In [ ]:
import os

os.environ["POLYTOPE_USERNAME"] = "xxx"
os.environ["POLYTOPE_PASSWORD"] = "xxx"
os.environ["POLYTOPE_ADDRESS"] = "https://polytope-dev.mchml.cscs.ch"

2. **Querying Data**

The below query will only work if the previous notebook was successfully run (in full) by the designated service account at CSCS which has permissions to write back the ECHOTOPinM field to FDB.

In [ ]:
from datetime import datetime, timedelta

# Current time
now = datetime.now()

# Subtract 12 hours
past_time = now - timedelta(hours=12)

# Round down to the nearest multiple of 3
rounded_hour = (past_time.hour // 3) * 3
rounded_time = past_time.replace(hour=rounded_hour, minute=0, second=0, microsecond=0)

# Format as YYYYMMDD and HHMM
date = rounded_time.strftime('%Y%m%d')
time = rounded_time.strftime('%H%M')
date,time

In [ ]:
from meteodatalab import mars, mch_model_data

request = mars.Request(
    param="T_2M",
    date=date,
    time=time,
    type="pf",
    number=1,
    step=0,
    levtype=mars.LevType.SURFACE,
    model=mars.Model.ICON_CH1_EPS,
)

In [ ]:
import earthkit.data as ekd
ds = ekd.from_source(
    "polytope",
    "mch",
    request.to_polytope(),
    stream=False
).to_xarray()

In [ ]:
from earthkit.plots import Map
from earthkit.plots.geo import bounds, domains
import cartopy.crs as ccrs

xmin, xmax = 5.5, 11   # Longitude bounds
ymin, ymax = 45.5, 48   # Latitude bounds

bbox = bounds.BoundingBox(xmin, xmax, ymin, ymax, ccrs.Geodetic())
domain = domains.Domain.from_bbox(
    bbox=bbox,
    name="CH2"
)
chart = Map(domain=domain)
chart.point_cloud(ds["T_2M"], x="longitude", y="latitude")

chart.coastlines()
chart.borders()
chart.gridlines()


chart.title("{long_name}")

chart.legend()

chart.show()